## Kaggle

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Saving Submission file Using Best Model</p>


### <p style="background-color: #fdefff;color:#c12eff;margin: 0;display: inline-block;padding:.4rem;border-radius:.5rem;border: 1px solid #c059ff">Please Upvote if you Really liked this</p>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Bibliotecas

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score, precision_score,roc_auc_score,confusion_matrix, classification_report

## Dados

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
exemplo = pd.read_csv("../input/titanic/gender_submission.csv")
respostacorreta = pd.read_csv('https://raw.githubusercontent.com/wallinsonoliveira/competicoes-kaggle/refs/heads/main/Titanic/respostacorreta.csv', na_values='?')
test["Survived"] = respostacorreta["Survived"]
#print(train.shape)
#print(test.shape)
#test.head()

#Train - PassengerId: 1 a 891
#Test - PassengerId: 892 a 1309

## Análise de Dados 1

In [ ]:
train

In [ ]:
train.isnull().sum()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train)

In [ ]:
filtered_data = train[~train['Embarked'].isin(['C', 'S', 'Q'])]
filtered_data

In [ ]:
sns.displot(data=train,x='Embarked')

In [ ]:
sns.countplot(x='Survived',data=train,hue='Sex',palette='RdBu_r')

In [ ]:
sns.countplot(x='Survived',data=train,hue='Pclass')

In [ ]:
sns.displot(train['Age'],kde=False,bins=30)

In [ ]:
survived_fares = train[train['Survived'] == 1]['Fare']
not_survived_fares = train[train['Survived'] == 0]['Fare']

plt.figure(figsize=(10, 6))
plt.hist([survived_fares, not_survived_fares], bins=40, color=['green', 'red'], label=['Survived', 'Not Survived'])
plt.xlabel('Fare')
plt.ylabel('Frequencia')
plt.title('Distribuição de Taxas por Sobreviventes')
plt.legend()
plt.show()

In [ ]:
sns.countplot(x='SibSp',data=train,hue='Survived',palette='RdBu_r')

In [ ]:
sns.countplot(x='Parch',data=train,hue='Survived',palette='RdBu_r')

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cmap=('viridis'))

In [ ]:
sns.heatmap(test.isnull(),yticklabels=False,cmap=('viridis'))

## Tratamento de Dados

In [ ]:
import pandas as pd
import numpy as np

# Copiar dados de treino e teste - original# Copiar dados de treino e teste
dados_train = train.copy()
dados_test = test.copy()

# Concatenar dados de treino e teste para processar ambos de uma vez
concat = pd.concat([dados_train, dados_test], axis=0, ignore_index=True)

# Correção de erros na base
concat['Parch'] = np.where(concat['Name'].isin(['Newell, Miss. Madeleine', 'Newell, Miss. Marjorie', 'Ware, Mrs. John James (Florence Louise Long)', 'Ware, Mr. John James', 'Watt, Mrs. James (Elizabeth "Bessie" Inglis Milne)', 'Watt, Miss. Bertha J']), 1, concat['Parch'])
concat['Parch'] = np.where(concat['Name']=='Natsch, Mr. Charles H', 0, concat['Parch'])
concat['SibSp'] = np.where(concat['Name'].isin(['Risien, Mrs. Samuel (Emma)', 'Risien, Mr. Samuel Beard']), 1, concat['SibSp'])


#concat['Sobrenome'] = concat['Name'].apply(lambda x: x.split(',')[0].strip())
#sobrenomes_unicos = concat['Sobrenome'].unique()
#categorias = {sobrenome: chr(65 + i) for i, sobrenome in enumerate(sobrenomes_unicos)}
#concat['CategoriaSobrenome'] = concat['Sobrenome'].map(categorias)

# Identifica a "moda" (valor mais comum) da coluna `Embarked`
embarked_mode = concat['Embarked'].mode()[0]

# Preenche valoes NaN da coluna `Embarked com a "moda"
concat['Embarked'] = concat['Embarked'].fillna(embarked_mode)

# Cria uma coluna T_partner com a soma dos familiares a bordo
concat['T_partner'] = concat["SibSp"]+train["Parch"]

# Cria uma coluna Alone que indica se o passageiro estava sozinho ou não
concat['Alone'] = np.where(concat['T_partner']>0, 0, 1)

# Extrai do nome o título do passageiro e depois converte para números de acordo o título
concat['Words_Count'] = concat['Name'].apply(lambda x: len(x.split()))
concat['Title'] = concat.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
concat['Title'] = concat['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
concat['Title'] = concat['Title'].replace('Mlle', 'Miss')
concat['Title'] = concat['Title'].replace('Ms', 'Miss')
concat['Title'] = concat['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
concat['Title'] = concat['Title'].map(title_mapping)
concat['Title'] = concat['Title'].fillna(0)

#Preencher Age nulas com a média das idades - Tentar preencher com a média por sexo também
#concat['Age_Was_Null'] = concat['Age'].isnull()  # Identificar valores nulos antes do preenchimento
#concat['Age'] = pd.to_numeric(concat['Age'], errors='coerce')
#concat['Age'] = concat['Age'].fillna(concat['Age'].median())

# Preenche idades nulas com a média das idades por `Title`
mean_age_per_title = concat.groupby('Title')['Age'].transform('mean')
concat['Age'] = concat['Age'].fillna(mean_age_per_title)

# Substitui dados nulos da cabine por U (Unknown)
concat['Cabin'] = concat['Cabin'].fillna('U')

import re
# Extrai primeira letra da cabine
concat['Cabin'] = concat['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
cabin_category = {'A':9, 'B':8, 'C':7, 'D':6, 'E':5, 'F':4, 'G':3, 'T':2, 'U':1}

# Substitui letras por numeros de acordo com a categoria, onde U é desconhecido
concat['Cabin'] = concat['Cabin'].map(cabin_category)

# Cria uma coluna is_minor com 1 se o passageiro tiver menos de 16 anos
concat['is_minor']=np.where(concat['Age']<=16,1,0)

# Limpa tickets. Pega os 3 primeiros dígitos do ticket, converte para categorias e depois para números
concat['Ticket_type'] = concat['Ticket'].apply(lambda x: x[0:3])
concat['Ticket_type'] = concat['Ticket_type'].astype('category')
concat['Ticket_type'] = concat['Ticket_type'].cat.codes

# Preencher valores nulos da coluna `Fare` com a média da coluna `Fare` dos registros da classe relacionada à linha que a coluna `Fare` está NaN
mean_fare_by_pclass = concat.groupby('Pclass')['Fare'].transform('mean')
concat['Fare'] = concat['Fare'].fillna(mean_fare_by_pclass)

# Apagar colunas extras
concat.drop(["T_partner", "Parch", "SibSp", "Name", "Ticket","Words_Count"], axis = 1, inplace = True)

# Transformar colunas categórias em numérias usando get_dummies
concat = pd.get_dummies(concat, drop_first=True)

# Filtrar a linha onde PassengerID é igual a 1044
#linha_1044 = concat[concat['PassengerId'] == 1044]
#print(linha_1044)

### BASE TREINAMENTO
#df = concat[concat['Survived'].notnull()].copy()
df = concat[concat['PassengerId'].between(1, 891)]
df = df.drop(columns=['PassengerId'])
df['Survived'] = df['Survived'].astype(int)

#Train - PassengerId: 1 a 891
#Test - PassengerId: 892 a 1309

### BASE TESTE - PARA PREVISÃO E SUBMISSÃO
#teste = concat[concat['Survived'].isnull()].copy()
teste = concat[concat['PassengerId'].between(892, 1309)]
teste_modelo_y = teste["Survived"]
teste_modelo = teste.drop(columns=['PassengerId', 'Survived']) # para ser usado no modelo
teste_previsao = teste.drop(columns=['Survived']) # para ser usado para submissão

print("feito")

In [ ]:
#concat.isnull().sum()
#concat.info()
concat.head()
#teste_modelo.info()
#teste_modelo_y.info()
#teste_previsao.info()
#res_y.info()
#print(concat.head(10))

## Análise de Dados 2

In [ ]:
sns.heatmap(concat.isnull(),yticklabels=False,cmap=('viridis'))

In [ ]:
"""
#sns.countplot(x='Title',data=concat,hue='Age',palette='RdBu_r')
mean_age_per_title = concat.groupby('Title')['Age'].mean().reset_index()

# Plota a média de idade por título
plt.figure(figsize=(10, 6))
sns.barplot(x='Title', y='Age', data=mean_age_per_title, palette='RdBu_r')
plt.ylabel('Média de Idade')
plt.title('Média de Idade por Título')
plt.show()
"""

In [ ]:
plt.figure(figsize=(30, 3))
corr_matrix = concat.drop(columns=['PassengerId']).corr()
sns.heatmap(corr_matrix.iloc[0:1], annot=True, cmap="coolwarm")
plt.show()
#corr_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configura o tamanho da figura
plt.figure(figsize=(15, 10))

# Calcula e plota a matriz de correlação completa, ignorando a coluna 'PassengerId'
corr_matrix = concat.drop(columns=['PassengerId']).corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)

# Exibe o gráfico
plt.show()


## Pre-processamento dos Dados

In [ ]:
#train_res = train["Survived"].reset_index(drop = True)

# Separa a coluna alvo 'Survived' em y
y = df['Survived']

# Separa outras colunas em X
X = df.drop(columns=['Survived'])

# Dividir dataset em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
print("feito")

## Busca por Hiperparâmetros

### Busca Bayesiana

In [ ]:
"""
# HyperOpt - Busca Bayesiana

#!pip install hyperopt

from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale, normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import logging
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import catboost as catboost
import logging
from sklearn.model_selection import cross_val_score, GridSearchCV
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

# Modelos - aqui são as classes, não as instâncias já criadas
models = {
    'logistic_regression': LogisticRegression,
    'rf': RandomForestClassifier,
    'knn': KNeighborsClassifier,
    'svc': SVC,
    'decision_tree': DecisionTreeClassifier,
    'xgboost': XGBClassifier,
    'catboost': CatBoostClassifier,
    'lgbm': LGBMClassifier,
    'bagging': BaggingClassifier
}

base_X_train = X.copy()
base_y_train = y.copy()

def search_space(model):
    model = model.lower()
    space = {}
    if model == 'bagging':
        space = {
            #'n_estimators': hp.choice('n_estimators', range(10, 200)),  # Número de estimadores
            'max_samples': hp.uniform('max_samples', 0.1, 1.0),  # Proporção do conjunto de dados
            'max_features': hp.uniform('max_features', 0.1, 1.0),  # Proporção de características
            'bootstrap': hp.choice('bootstrap', [True, False]),  # Amostragem com reposição
            'base_estimator': hp.choice('base_estimator', [
                LogisticRegression(),  # Corrigido: agora passando a instância
                #RandomForestClassifier(),
                DecisionTreeClassifier()
            ])  # Instâncias de modelos de base para o Bagging
        }
    elif model == 'catboost':
        space = {
            'iterations': hp.choice('iterations', range(100,102)),
            'depth': hp.choice('depth', range(3, 10)),
            'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
            'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
            'border_count': hp.choice('border_count', range(32, 255))
        }
    elif model == 'lgbm':
        space = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth': hp.choice('max_depth', range(3, 15)),
            'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
            'num_leaves': hp.choice('num_leaves', range(20, 150)),
            'feature_fraction': hp.uniform('feature_fraction', 0.5, 1.0)
        }
    elif model == 'knn':
        space = {
            'n_neighbors': hp.choice('n_neighbors', range(3, 100)),
            'p': hp.choice('p', range(1, 20))
            #'scale': hp.choice('scale', [0, 1]),
            #'normalize': hp.choice('normalize', [0, 1]),
        }
    elif model == 'svc':
      space = {
          'C': hp.uniform('C', 0, 1000),
          #'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
          'gamma': hp.choice('gamma', ['auto', 'scale']),  # Substitua 'RS*' por 'scale'
          'class_weight': hp.choice('class_weight', ['balanced', None])  # Corrigido de 'kernel' para 'class_weight'
      }

    elif model == 'logistic_regression':
        space = {
            #'fit_intercept': hp.choice('fit_intercept', [True, False]),
            #'C': hp.uniform('C', 0.05, 3),
            'C': hp.uniform('C', 0.05, 100),
            'solver': hp.choice('solver', ['liblinear']),
            #'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
            #'max_iter': hp.choice('max_iter', range(2, 1000)),
            #'multi_class': 'auto',
            #'class_weight': 'balanced'
            'penalty': hp.choice('penalty',['l1','l2'])
        }

    elif model == 'rf':
        space = {
            'max_depth': hp.choice('max_depth', range(1, 200)),
            'min_samples_split': hp.choice('min_samples_split', range(2, 40)),
            'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 20)),
            'max_features': hp.choice('max_features', range(1, 3)),
            'n_estimators': hp.choice('n_estimators', range(50, 1200))
        }

    elif model == 'decision_tree':  # Adicionando Decision Tree
        space = {
            'criterion': hp.choice('criterion', ['gini', 'entropy']),
            'max_depth': hp.choice('max_depth', range(1, 100)),
            'min_samples_split': hp.choice('min_samples_split', range(2, 100)),
            'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 20)),
            'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
        }

    elif model == 'xgboost':  # Adicionando espaço de busca para o XGBoost
        space = {
            'eta': hp.choice('eta', [0.01, 0.015, 0.025, 0.05, 0.1]),
            'gamma': hp.choice('gamma', [0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]),
            'max_depth': hp.choice('max_depth', [3, 5, 7, 9, 12, 15, 17, 25]),
            'min_child_weight': hp.choice('min_child_weight', [1, 3, 5, 7]),
            'subsample': hp.choice('subsample', [0.6, 0.7, 0.8, 0.9, 1.0]),
            'colsample_bytree': hp.choice('colsample_bytree', [0.6, 0.7, 0.8, 0.9, 1.0]),
            'lambda': hp.choice('lambda', [0.01, 0.1, 1.0]),  # RS* não é suportado diretamente
            'alpha': hp.choice('alpha', [0, 0.1, 0.5, 1.0])   # RS* removido para evitar erro
         }

    space['model'] = model
    return space

def get_acc_status(clf, X_, y):
    acc = cross_val_score(clf, X_, y, cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

def scale_normalize(params, X_):
    if params.get('scale', 0):
        X_ = scale(X_)
    if params.get('normalize', 0):
        X_ = normalize(X_)
    return X_

def obj_fnc(params):
    model_name = params.get('model').lower()
    X_ = scale_normalize(params, base_X_train[:])  # Agora usando X_train
    del params['model']

    # Instanciar o modelo com os parâmetros ajustados
    clf = models[model_name](**params)

    return get_acc_status(clf, X_, base_y_train)  # Agora usando y_train

hypopt_trials = Trials()

# Executando a busca bayesiana
best_params = fmin(
    fn=obj_fnc,
    space=search_space('bagging'),  # Ajuste aqui o modelo desejado ('knn', 'svc', etc.)
    algo=tpe.suggest,
    max_evals=1000,
    trials=hypopt_trials
)

print(best_params)
print(hypopt_trials.best_trial['result']['loss'])
"""

### Grid Search

In [ ]:
"""
# Grid Search - SVM
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale, normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import logging
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import catboost as catboost
import logging
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

base_X_train = X.copy()
base_y_train = y.copy()

svm = SVC(random_state=100)

param_grid = {
    'C': list(range(1, 500, 10)),                # 1 a 100 de 5 em 5 passos
    #'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],             # Função de kernel a ser usada
    #'degree': [1, 2, 3, 4],                                     # Grau do polinômio se o kernel for 'poly'
    #'gamma': ['scale', 'auto', 0.01, 0.1, 1],                  # Coeficiente do kernel
    'coef0': [0.0, 0.5, 1.0],                                   # Termo independente no núcleo
    'shrinking': [True, False],                                 # Habilita ou desabilita o encolhimento
    #'class_weight': [None, 'balanced']                         # Peso das classes
    'tol': [1e-4, 1e-3, 1e-2],                                 # Tolerância para critério de parada
    #'max_iter': [100, 500]                                 # Número máximo de iterações
}

#Melhores hiperparâmetros: {'C': 81, 'class_weight': 'balanced', 'degree': 1, 'gamma': 'scale'}
#F1-Score:  0.71378350410412
#Melhores hiperparâmetros: {'C': 201, 'class_weight': 'balanced', 'degree': 1, 'gamma': 'scale'}
#F1-Score:  0.7249733335690588

# Realizar a busca de hiperparâmetros com validação cruzada
grid_search = GridSearchCV(svm, param_grid, cv=3, n_jobs=-1, scoring='f1')
grid_search.fit(base_X_train, base_y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Exibir os melhores hiperparâmetros e o F1-Score
print("Melhores hiperparâmetros:", grid_search.best_params_)
print("F1-Score: ", grid_search.best_score_)
"""

In [ ]:
"""
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Supondo que base_X_train e base_y_train sejam os seus dados de treinamento
base_X_train = X.copy()
base_y_train = y.copy()

# Inicializando o modelo KNN
knn = KNeighborsClassifier()

# Grade de parâmetros para otimização
param_grid = {
    'n_neighbors': list(range(1, 50, 5)),        # Número de vizinhos (de 1 a 50 com passo de 5)
    'weights': ['uniform', 'distance'],          # Peso uniforme ou por distância
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Métrica de distância
    'p': [1, 2]                                  # Parâmetro de potência da métrica Minkowski (1 para manhattan, 2 para euclidean)
}

# Configuração da busca em grade com validação cruzada
grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring='accuracy',       # Critério de avaliação, pode ser ajustado conforme necessário
    cv=5,                     # Número de folds na validação cruzada
    verbose=1,                # Nível de detalhamento
    n_jobs=-1                 # Usa todos os processadores disponíveis
)

# Realizando a busca
grid_search.fit(base_X_train, base_y_train)

# Melhor combinação de parâmetros encontrada
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Melhores parâmetros:", best_params)
print("Melhor acurácia:", best_score)
"""

In [ ]:
"""
# Decison Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Supondo que base_X_train e base_y_train sejam os seus dados de treinamento
base_X_train = X.copy()
base_y_train = y.copy()

# Inicializando o modelo de Árvore de Decisão
dtree = DecisionTreeClassifier()

# Grade de parâmetros para otimização
param_grid = {
    'criterion': ['gini', 'entropy','log_loss'],  # Critério de divisão da árvore
    'max_depth': [None] + list(range(18, 21)),  # Profundidade máxima da árvore (None para crescimento ilimitado)
    'min_samples_split': [2] + list(range(5, 15)),  # Número mínimo de amostras necessárias para dividir um nó
    'min_samples_leaf': [1] + list(range(1, 11)),   # Número mínimo de amostras que um nó folha deve ter
    'max_features': [0,1,2]  # Número de recursos a considerar ao procurar a melhor divisão
}
#    {'criterion': 0, 'max_depth': 19, 'max_features': 2, 'min_samples_leaf': 7, 'min_samples_split': 19}

# Configuração da busca em grade com validação cruzada
grid_search = GridSearchCV(
    estimator=dtree,
    param_grid=param_grid,
    scoring='accuracy',       # Critério de avaliação
    cv=5,                     # Número de folds na validação cruzada
    verbose=1,                # Nível de detalhamento
    n_jobs=-1                 # Usa todos os processadores disponíveis
)

# Realizando a busca
grid_search.fit(base_X_train, base_y_train)

# Melhor combinação de parâmetros encontrada
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Melhores parâmetros:", best_params)
print("Melhor acurácia:", best_score)
"""

## Algoritmos - Teste Automático

In [ ]:
# Algoritmos de aprendizagem
import logging
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import catboost as catboost
import logging
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

# Configurando o logger do LightGBM
lgb_logger = logging.getLogger("lightgbm")
lgb_logger.setLevel(logging.ERROR)

nc = 5
base_X_train = X.copy()
base_y_train = y.copy()
#teste_modelo = pd.get_dummies(teste, drop_first=True)
#teste_modelo = teste_modelo.reindex(columns=X.columns, fill_value=False)

resultados_df = pd.DataFrame()

# Modelos de Machine Learning

estimators = [
    ('rf', RandomForestClassifier(max_depth=20,max_features=1,min_samples_leaf=1,min_samples_split=4,n_estimators=1010,random_state=42)),
    ('gb', GradientBoostingClassifier()),
    ('et', ExtraTreesClassifier()),
    ('catbost', CatBoostClassifier(verbose=0)),
    ('reglog',LogisticRegression(C= 3,penalty= 'l2',solver= 'liblinear')),
    ('xgb',XGBClassifier()),
    ('SVM', SVC(probability=True,C=441,class_weight='balanced',coef0=0,shrinking=True,tol=0.01,gamma='scale',degree=1))
]

modelos = {
    'Regressão Logística': LogisticRegression(C= 3,penalty= 'l2',solver= 'liblinear'),
    'KNN': KNeighborsClassifier(n_neighbors=16,p=1,weights='distance',metric='manhattan'),
    'Decision Tree': DecisionTreeClassifier(criterion='entropy',max_depth=18,max_features=2,min_samples_leaf=4,min_samples_split=11),
    'Random Forest': RandomForestClassifier(max_depth=20,max_features=1,min_samples_leaf=1,min_samples_split=4,n_estimators=1010,random_state=42),
    'SVM': SVC(probability=True,C=441,class_weight='balanced',coef0=0,shrinking=True,tol=0.01,gamma='scale',degree=1),
    'Naive Bayes': GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'Bagging': BaggingClassifier(),
    'Perceptron': Perceptron(),
    'MLP Classifier': MLPClassifier(max_iter=1000),
    'QDA': QuadraticDiscriminantAnalysis(),
    'XGBoost': XGBClassifier(),
    'LightGBM': LGBMClassifier(verbosity=-1),
    'CatBoost': CatBoostClassifier(verbose=0),
    'Voting Classifier': VotingClassifier(estimators=[
       ('rf', RandomForestClassifier(max_depth=20,max_features=1,min_samples_leaf=1,min_samples_split=4,n_estimators=1010,random_state=42)),
       ('gb', GradientBoostingClassifier()),
       ('et', ExtraTreesClassifier()),
       ('catbost', CatBoostClassifier(verbose=0)),
       ('reglog',LogisticRegression(C= 3,penalty= 'l2',solver= 'liblinear')),
       ('xgb',XGBClassifier()),
       ('SVM', SVC(probability=True,C=441,class_weight='balanced',coef0=0,shrinking=True,tol=0.01,gamma='scale',degree=1))
    ], voting='soft'),
    'Stacking': StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
}

# Cross-Validation e avaliação para cada modelo individual
for nome, modelo in modelos.items():
    # Fit e Cross-Validation com 3 folds
    modelo.fit(base_X_train, base_y_train)
    cv_result = cross_val_score(modelo, base_X_train, base_y_train, cv=nc, scoring='accuracy')

    # Previsões no conjunto de teste
    #y_pred = modelo.predict(X_test)
    y_pred = modelo.predict(teste_modelo)
    
    # Calcula métricas no conjunto de teste
    acuracia = accuracy_score(teste_modelo_y, y_pred)
    f1_positivo = f1_score(teste_modelo_y, y_pred, pos_label=1)
    f1_negativo = f1_score(teste_modelo_y, y_pred, pos_label=0)

    # Confusion matrix para extrair TP, TN, FP, FN
    tn, fp, fn, tp = confusion_matrix(teste_modelo_y, y_pred).ravel()
    erros = fp + fn

    # Cria uma nova linha com os resultados
    nova_linha = pd.DataFrame([{
        'Modelo': nome,
        'CV': cv_result.mean() * 100,
        'DP': cv_result.std() * 100,
        'Acurácia': acuracia * 100,
        'F1 +': f1_positivo * 100,
        'F1 -': f1_negativo * 100,
        'VP': tp,
        'VN': tn,
        'FP': fp,
        'FN': fn,
        'Erros': erros
    }])

    # Adiciona a nova linha ao DataFrame
    resultados_df = pd.concat([resultados_df, nova_linha], ignore_index=True)

resultados_df = resultados_df.round(2)

# Ordena o DataFrame 'resultados_df' pela coluna desejada (exemplo: 'Acurácia (X_test)') em ordem decrescente
resultados_df = resultados_df.sort_values(by='Erros', ascending=True)

from IPython.display import display, HTML

# Converte o DataFrame para HTML com alinhamento centralizado
display(HTML(f"<div style='display: flex; justify-content: center;'>{resultados_df.to_html(index=False)}</div>"))


## Importância das Variáveis por Permutação

In [ ]:
"""
from sklearn.inspection import permutation_importance
from tqdm import tqdm
import pandas as pd

#xgboost = XGBClassifier(objective='multi:softmax', num_class=2, random_state=42).fit(base_X_train, base_y_train)
xgboost = XGBClassifier().fit(base_X_train, base_y_train)
catboost = CatBoostClassifier(verbose=0).fit(base_X_train, base_y_train)

modeloescolhido = catboost
repeticoes = 30

# Função para exibir progresso ao fazer o permutation_importance
def permutation_importance_with_progress(model, X, y, n_repeats, random_state, n_jobs):
    # Inicializa a barra de progresso com n_repeats iterações
    with tqdm(total=n_repeats, desc="Calculando importância por permutação", unit="iteração") as pbar:

        # Define uma função de callback que atualiza a barra de progresso após cada repetição
        def update_progress_bar(_):
            pbar.update(1)

        # Executa o permutation_importance
        perm_importance = permutation_importance(
            model, X, y, n_repeats=n_repeats, random_state=random_state, n_jobs=n_jobs
        )

        # Atualiza a barra de progresso ao final
        pbar.update(n_repeats)

    return perm_importance

# Chamar a função para calcular a importância com a barra de progresso
perm_importance = permutation_importance_with_progress(modeloescolhido, X_test, y_test, n_repeats=repeticoes, random_state=42, n_jobs=-1)

# Organizar os resultados em um DataFrame
feature_names = X_test.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': perm_importance.importances_mean})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibir as 10 features mais importantes
print(feature_importances)
"""

In [ ]:
"""
import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

modeloescolhido = xgboost
repeticoes = 30

# Calcular a importância das features por permutação
perm_importance = permutation_importance(modeloescolhido, X_test, y_test, n_repeats=repeticoes, random_state=42, n_jobs=-1)

# Obter as importâncias médias e o desvio padrão das importâncias
importance_vals = perm_importance.importances_mean
importance_std = perm_importance.importances_std
features = X_test.columns

# Certifique-se de que os arrays de importâncias e as features tenham o mesmo tamanho
if len(importance_vals) != len(features):
    raise ValueError("O número de importâncias não corresponde ao número de features. Verifique se X_test está correto.")

# Ordenar as importâncias em ordem decrescente
indices = np.argsort(importance_vals)[::-1]

# Escolher o número mínimo de features entre o solicitado (15) e o total disponível
top_n = min(15, len(importance_vals))
top_indices = indices[:top_n]

# Plotar a importância das features mais importantes
plt.figure(figsize=(10, 6))
plt.title("Importância das Features por Permutação - XGBoost")
plt.barh(range(top_n), importance_vals[top_indices], xerr=importance_std[top_indices], align="center", color='skyblue')
plt.yticks(range(top_n), features[top_indices])
plt.gca().invert_yaxis()  # Inverter o eixo Y para que a feature mais importante fique no topo
plt.xlabel('Grau de Importância')
plt.ylabel('Features')
plt.tight_layout()
plt.show()
"""

## Algorítmos - Teste Manual

In [ ]:
"""from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

bagging_model = BaggingClassifier()

# Criando um modelo de Bagging com árvores de decisão
#bagging_model = BaggingClassifier(
#    base_estimator=RandomForestClassifier(max_depth=20,max_features=1,min_samples_leaf=1,min_samples_split=4,n_estimators=1010,random_state=42),
#    n_estimators=100,  # Número de modelos (árvores) a serem treinados
#    random_state=42,
#    max_features=0.55,
#    max_samples=0.65,
#    bootstrap=1
#)

#{'base_estimator': 1, 'bootstrap': 1, 'max_features': 0.5514091541886079, 'max_samples': 0.6536918434137314}


# Treinando o modelo
bagging_model.fit(X, y)

# Fazendo previsões e avaliando a acurácia
y_pred = bagging_model.predict(teste_modelo)
print("Acurácia:", accuracy_score(teste_modelo_y, y_pred))
"""

In [ ]:
"""
# Algoritmos de aprendizagem
import logging
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import catboost as catboost
import logging
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

base_X_train = X.copy()
base_y_train = y.copy()

modelo_previsao = XGBClassifier().fit(base_X_train, base_y_train)

#ensemble
xgb_clf = XGBClassifier()
xgb_clf.fit(base_X_train, base_y_train,eval_metric=["auc", "logloss"],verbose=True)

from lightgbm import LGBMClassifier
import lightgbm as lgb

lgb_clf = LGBMClassifier()
lgb_clf.fit(base_X_train, base_y_train)

from catboost import CatBoostClassifier
import catboost as catboost

cat_clf = CatBoostClassifier(verbose=0)
cat_clf.fit(base_X_train, base_y_train)

from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(max_depth=20,
    criterion="entropy",
    min_samples_leaf=14,
    min_samples_split=5) 
decision_tree.fit(base_X_train, base_y_train)

from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=200)
random_forest.fit(base_X_train, base_y_train)

from sklearn.linear_model import LogisticRegression
reglog = LogisticRegression(max_iter=1000).fit(base_X_train, base_y_train)
"""

## Submeter

In [ ]:
base_X_train = X.copy()
base_y_train = y.copy()
"""
estimators = [
    ('rf', RandomForestClassifier(max_depth=20,max_features=1,min_samples_leaf=1,min_samples_split=4,n_estimators=1010,random_state=42)),
    ('gb', GradientBoostingClassifier()),
    ('et', ExtraTreesClassifier()),
    ('catbost', CatBoostClassifier(verbose=0)),
    ('reglog',LogisticRegression(C= 3,penalty= 'l2',solver= 'liblinear')),
    ('xgb',XGBClassifier()),
    ('SVM', SVC(probability=True,C=441,class_weight='balanced',coef0=0,shrinking=True,tol=0.01,gamma='scale',degree=1))
]
"""
#stacking = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(max_depth=20,max_features=1,min_samples_leaf=1,min_samples_split=4,n_estimators=1010,random_state=42)).fit(base_X_train, base_y_train)

#from sklearn.linear_model import LogisticRegression
#reglog = LogisticRegression(max_iter=1000).fit(base_X_train, base_y_train)

#from catboost import CatBoostClassifier
#import catboost as catboost
#catboost = CatBoostClassifier(verbose=0).fit(base_X_train, base_y_train)

decisiontree = DecisionTreeClassifier(criterion='entropy',max_depth=18,max_features=2,min_samples_leaf=4,min_samples_split=11).fit(base_X_train, base_y_train)

#teste_modelo = pd.get_dummies(teste, drop_first=True)
#teste_modelo = teste_modelo.reindex(columns=X.columns, fill_value=False)

teste_previsao['Survived'] = decisiontree.predict(teste_modelo)

resultado = teste_previsao[['PassengerId', 'Survived']]
resultado['Survived'] = resultado['Survived'].astype(int)
#resultado.loc[:, 'Survived'] = resultado['Survived'].astype(int)

#print(resultado)

#resultado.isnull().sum()

resultado.to_csv('submission.csv', index=False)

#from google.colab import files
#files.download('submission.csv')

#submit.to_csv(filename, index=False)
#print(resultado)


## Verificar Resultado

In [ ]:
"""
comparacao = pd.merge(resultado, respostacorreta, on='PassengerId', suffixes=('_predito', '_correto'))

# Calcular o número de acertos
acertos = (comparacao['Survived_predito'] == comparacao['Survived_correto']).sum()

# Calcular a porcentagem de acertos
porcentagem_acertos = (acertos / len(comparacao)) * 100

# Exibir os resultados
print(f"Número de acertos: {acertos}")
print(f"Porcentagem de acertos: {porcentagem_acertos:.2f}%")

#print(comparacao)
#print(len(comparacao))
"""